In [3]:
import pandas as pd
import requests
import csv
import time
import datetime
import config as config

In [36]:
# increase cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [54]:
url = "https://developer.cumtd.com/api/v2.2/json/getdeparturesbystop?key=" + config.mtd_api_key + "&stop_id=iu:1&count=5"
payload= {}
headers = {}
response = requests.request("GET", url, headers=headers, data=payload)

print(response.json()["time"])
print(response.json()["departures"][0]["stop_id"])
print(response.json()["departures"][0]["headsign"])
print(response.json()["departures"][1]["trip"]["direction"])
print(response.json()["departures"][0]["location"])
print("Sceduled: " + str(response.json()["departures"][1]["scheduled"]))
print("Expected: " + str(response.json()["departures"][1]["expected"]))
print(str(response.json()["departures"][1]["expected_mins"]) + " mins")

2021-11-13T16:48:10-06:00
IU:1
20U Red
North
{'lat': 40.11172, 'lon': -88.228917}
Sceduled: 2021-11-13T16:48:00-06:00
Expected: 2021-11-13T16:49:12-06:00
1 mins


In [10]:
# the time it says the bus will arrive vs. when it actually arrived

# check scheduled vs. expected time -> initial expected time difference
# then, send requests for current time as expected time gets closer to 0 --> -2 < 0 < 5
# if it goes beyond 5, stop sending requests, call this every 3 seconds

# bus scheduled and expected randomly get pushed back!!!

# add pt parameter, between route and count

# 2021-10-23T18:05:36-05:00 -> 18:05:36 for 'soft', datetime for 'hard'
def time_converter(time, soft_hard):
    split_1 = time.split("T")
    split_2 = split_1[1].split("-")
    if soft_hard == "hard":
        split_3 = split_2[0].split(':')
        return datetime.datetime.now().replace(hour = int(split_3[0]), minute = int(split_3[1]), second = int(split_3[2]), microsecond = 0)
    elif soft_hard == "soft":
        return str(split_2[0])

# Returns a list of all buses, with different vehicle_id's, currently heading to this stop_id.
# When a bus leaves, they stop appearing in requests, and therefore the list everytime it's called.
def arrival_queue(stop_id):
    list = []
    response = requests.get("https://developer.cumtd.com/api/v2.2/json/getdeparturesbystop?key=" + config.mtd_api_key_2 + "&stop_id=" + stop_id)
    for i in range(0, len(response.json()["departures"])):
        if str(response.json()["departures"][i]["headsign"]) not in list: # Eliminating any dupes, leaving them for next time they're called
            list.append(str(response.json()["departures"][i]["headsign"]))
    print("Bus arrival queue: ")
    print(list)
    return list

#  key:'headsign', 0:'route_id', 1:'origin', 2:'destination', 3:'scheduled', 4:'expected', 5:'expected_mins', dictionary with info about current buses with same stop_id
def stop_id_info(stop_id):
    bus_dict = {}
    response = requests.get("https://developer.cumtd.com/api/v2.2/json/getdeparturesbystop?key=" + config.mtd_api_key_2 + "&stop_id=" + stop_id)
    for i in range(0, len(response.json()["departures"])):
        if str(response.json()["departures"][i]["headsign"]) not in bus_dict.keys(): # Eliminating any dupes, leaving them for next time they're called
            bus_dict[str(response.json()["departures"][i]["headsign"])] = [str(response.json()["departures"][i]["route"]["route_id"]), str(response.json()["departures"][i]["origin"]["stop_id"]), str(response.json()["departures"][i]["destination"]["stop_id"]), str(response.json()["departures"][i]["scheduled"]), str(response.json()["departures"][i]["expected"]), str(response.json()["departures"][i]["expected_mins"])]
    return bus_dict
    print(bus_dict) # Eliminate this print later

# Returns time difference when given stop_id, the bus_headsign, and the index that determines whether it is scheduled or expected
def time_difference(stop_id, bus_headsign, index):
    response = requests.get("https://developer.cumtd.com/api/v2.2/json/getdeparturesbystop?key=" + config.mtd_api_key_2 + "&stop_id=" + stop_id)
    current_time = time_converter(response.json()["time"], 'hard')
    current_sc_ex = time_converter(stop_id_info(stop_id)[bus_headsign][index], 'hard')
    return (current_time - current_sc_ex).total_seconds()
                
# Outcome of whether current several buses came before or after the scheduled or expected time -> 3 for scheduled, 4 or expected
def bulk_difference(stop_id, sc_ex):
    results = []
    bus_headsign = arrival_queue(stop_id)
    response = requests.get("https://developer.cumtd.com/api/v2.2/json/getdeparturesbystop?key=" + config.mtd_api_key_2 + "&stop_id=" + stop_id)
    for i in range(0, 1):
        original_expected = stop_id_info(stop_id)[bus_headsign[i]][4]
        while (True):
            try:
                value = int(time_difference(stop_id, bus_headsign[i], sc_ex))
                if arrival_queue(stop_id)[0] != bus_headsign[i]:
                    print('Break at: ' + str(results[len(results) - 1]))
                    break
            except KeyError:
                print('Break at: ' + str(results[len(results) - 1]))
                break
            if -120 <= int(time_difference(stop_id, bus_headsign[i], sc_ex)) and int(time_difference(stop_id, bus_headsign[i], sc_ex)) <= 420:
                try:
                    print("Current time difference: " + str(time_difference(stop_id, bus_headsign[i], sc_ex)))
                    print("Expected time: " + stop_id_info(stop_id)[bus_headsign[i]][4] + "\n")
                    results.append(int(time_difference(stop_id, bus_headsign[i], sc_ex)))
                except KeyError:
                    print("KeyError")
                time.sleep(3)
            else:
                print("Time until next bus arrives: " + str(time_difference(stop_id, bus_headsign[i], sc_ex)))
                print("Expected time: " + stop_id_info(stop_id)[bus_headsign[i]][4] + "\n")
                time.sleep(30)
    print(results)
        
# Outcome of whether the bus came before or after the scheduled or expected time
def indiv_difference(headsign, stop_id, sc_or_ex):
    print('')

# print(time_difference('IU:1', '220N Illini Limited', 4))
bulk_difference('IU:1', 4)

Bus arrival queue: 
['120E Teal Orchard Downs', '220N Illini', '130N Silver', '50E Green']
Bus arrival queue: 
['120E Teal Orchard Downs', '220N Illini', '130N Silver', '50E Green']
Time until next bus arrives: -314.0
Expected time: 2021-11-13T18:42:12-06:00

Bus arrival queue: 
['120E Teal Orchard Downs', '220N Illini', '130N Silver', '50E Green']
Time until next bus arrives: -283.0
Expected time: 2021-11-13T18:42:12-06:00

Bus arrival queue: 
['120E Teal Orchard Downs', '220N Illini', '130N Silver', '50E Green']
Time until next bus arrives: -252.0
Expected time: 2021-11-13T18:42:12-06:00

Bus arrival queue: 
['120E Teal Orchard Downs', '220N Illini', '130N Silver', '50E Green']
Time until next bus arrives: -221.0
Expected time: 2021-11-13T18:42:12-06:00

Bus arrival queue: 
['120E Teal Orchard Downs', '220N Illini', '130N Silver', '50E Green']
Time until next bus arrives: -190.0
Expected time: 2021-11-13T18:42:12-06:00

Bus arrival queue: 
['120E Teal Orchard Downs', '220N Illini', '

In [5]:
response = requests.get("https://developer.cumtd.com/api/v2.2/json/getdeparturesbystop?key=" + config.mtd_api_key_2 + "&stop_id=" + 'IU:1')
response.json()

{'time': '2021-11-13T18:24:43-06:00',
 'new_changeset': True,
 'status': {'code': 200, 'msg': 'ok'},
 'rqst': {'method': 'GetDeparturesByStop', 'params': {'stop_id': 'IU:1'}},
 'departures': [{'stop_id': 'IU:1',
   'headsign': '120E Teal Orchard Downs',
   'route': {'route_color': '006991',
    'route_id': 'TEAL SATURDAY',
    'route_long_name': 'Teal Saturday',
    'route_short_name': '120',
    'route_text_color': 'ffffff'},
   'trip': {'trip_id': '[@12.0.44194660@][1250946648828]/160__T2_UISA',
    'trip_headsign': 'Orchard Downs',
    'route_id': 'TEAL SATURDAY',
    'block_id': 'T2 UISA',
    'direction': 'East',
    'service_id': 'T2 UISA',
    'shape_id': '12E TEAL WEEKEND 13'},
   'vehicle_id': '1714',
   'origin': {'stop_id': 'IT:5'},
   'destination': {'stop_id': 'ODSS:1'},
   'is_monitored': True,
   'is_scheduled': True,
   'is_istop': True,
   'scheduled': '2021-11-13T18:22:00-06:00',
   'expected': '2021-11-13T18:25:43-06:00',
   'expected_mins': 1,
   'location': {'lat':